In [5]:
from buffer import Buffer
from node import Node

In [6]:
from src.channel import Channel
from src.range import Range
TOTAL_PACKAGES = 500

In [7]:
buffer_c = Buffer(capacity=TOTAL_PACKAGES)
node_c = Node(channels=[], buffer=buffer_c)

In [8]:
channel_b1c = Channel(range=Range(22, 28), receiver=node_c)
channel_b2c = Channel(range=Range(25, 25), receiver=node_c)

buffer_b = Buffer(capacity=25, threshold=20)

node_b1 = Node(channels=[channel_b1c], buffer=buffer_b)
node_b2 = Node(channels=[channel_b2c], buffer=buffer_b)

In [8]:
channel_ab1 = Channel(range=Range(20, 20), receiver=node_b1)
channel_ab2 = Channel(range=Range(15, 25), receiver=node_b2)

buffer_a = Buffer(capacity=20)

node_a = Node(channels=[channel_ab1, channel_ab2], buffer=buffer_a)